下載相關資料的小魔法

In [ ]:
!curl "https://gist.githubusercontent.com/penut85420/5b383ee875f66cfba70c46ad0e2dd21b/raw/fbd21d468add3ad99370e23ed9dcbf10aa7740c9/kana-spell.json" -o "kana-spell.json"

匯入相關套件

In [ ]:
# %pip install gradio -Uq
import json
import random

import gradio as gr

讀取 JSON 檔案

In [ ]:
def load_json(file_path):
    with open(file_path, "rt", encoding="UTF-8") as fp:
        return json.load(fp)


data = load_json("kana-spell.json")

準備資料

In [ ]:
data = load_json("kana-spell.json")

hiragana = data["hiragana"]
katakana = data["katakana"]
category = data["category"]
spell = data["spell"]

開始測驗時選擇題目

In [ ]:
def start_test(kana, seion, dakuon, handakuon, yoon):
    # seion 等參數會傳入像是 ["a", "ka"] 等列表
    # 使用 * 將這些列表展開成一維列表
    category = [*seion, *dakuon, *handakuon, *yoon]

    # 從平假名或片假名資料中取出對應的假名，全部放在 char_list 裡面
    char_list = list()
    char_list += [ch for k in category for ch in hiragana[k]] if "平假名" in kana else []
    char_list += [ch for k in category for ch in katakana[k]] if "片假名" in kana else []

    # 如果 char_list 是空的，則拋出錯誤
    if not char_list:
        raise gr.Error("請至少選擇一個類別")

    # 隨機打亂 char_list 的順序
    random.shuffle(char_list)

    # 取出第一個假名
    char = char_list.pop(0)

    # 回傳 char 給 txt_test 用來顯示題目
    # 第一個 char_list 回傳給 st_queue 用來紀錄題目狀態
    # 第二個 char_list 回傳給 debug 用來檢查
    # 回傳 gr.Tabs(selected=1) 來切換到測驗分頁
    return char, char_list, char_list, gr.Tabs(selected=1)

檢查作答是否正確

In [ ]:
def check_answer(txt_test, txt_input, n_correct, n_total, txt_records):
    # 將輸入的拼音轉為小寫並去除前後空白
    txt_input = str.lower(txt_input).strip()

    # 如果 txt_input 符合任何一種拼音，則正確
    if txt_input in spell[txt_test]:
        n_correct += 1
        txt_info = "正確！"

    # 如果拼音不正確，提示使用者正確的答案有哪些可能
    else:
        answer = ", ".join(spell[txt_test])
        txt_info = f"錯誤，正確答案為 {answer}"
        txt_records += f"題目：{txt_test}、正解：{answer}、輸入：{txt_input}\n"

    n_total += 1
    accuracy = n_correct / n_total

    # 回傳 None 來清空 txt_input 的內容
    return None, txt_info, n_correct, n_total, f"{accuracy:.2%}", txt_records

顯示下一題

In [ ]:
def next_char(st_queue, n_correct, n_total, txt_record):
    # 若 st_queue 是空的，則顯示測驗結束的訊息
    if not st_queue:
        gr.Info("測驗結束！")
        accuracy = n_correct / n_total
        txt_record += f"正確率 {accuracy:.2%} ({n_correct}/{n_total})"
        return None, None, None, txt_record, gr.Tabs(selected=2)

    # 繼續從 st_queue 中取出下一個假名
    char = list.pop(st_queue, 0)

    # 分別回傳給 txt_test, st_queue, debug 等元件
    return char, st_queue, st_queue, txt_record, gr.Tabs(selected=1)

全選與全不選

In [ ]:
def select_all():
    return (
        ["平假名", "片假名"],
        category["seion"],
        category["dakuon"],
        category["handakuon"],
        category["youon"],
    )


def select_none():
    return [], [], [], [], []

In [ ]:
def reset():
    return 0, 0, "100.00%", None, None


def back():
    return gr.Tabs(selected=0)

In [ ]:
font = gr.themes.GoogleFont("Kiwi Maru")
theme = gr.themes.Ocean(font=font)

with gr.Blocks(theme=theme) as app:
    st_queue = gr.State(None)

    # region define layout
    with gr.Tabs(selected=0) as tabs:
        with gr.Tab(label="設定", id=0):
            with gr.Group():
                chk_kana = gr.CheckboxGroup(["平假名", "片假名"], value=["平假名"], label="假名")
                chk_seion = gr.CheckboxGroup(category["seion"], value=["a"], label="清音")
                with gr.Row():
                    chk_dakuon = gr.CheckboxGroup(category["dakuon"], label="濁音")
                    chk_handakuon = gr.CheckboxGroup(category["handakuon"], label="半濁音")
                chk_youon = gr.CheckboxGroup(category["youon"], label="拗音")

            with gr.Row():
                btn_select_all = gr.Button("全選")
                btn_select_none = gr.Button("全不選")
            btn_start = gr.Button("開始測驗")

        with gr.Tab(label="測驗", id=1):
            with gr.Group():
                with gr.Row():
                    txt_test = gr.Textbox(label="題目", interactive=False)
                    txt_info = gr.Textbox(label="資訊", interactive=False)

                txt_input = gr.Textbox(label="作答", submit_btn=True)

                with gr.Row():
                    n_correct = gr.Number(label="答對題數", interactive=False)
                    n_total = gr.Number(label="總答題數", interactive=False)
                    txt_accuracy = gr.Text("100.00%", label="答對比率", interactive=False)

        with gr.Tab(label="紀錄", id=2):
            txt_records = gr.TextArea(show_label=False, interactive=False)
            btn_again = gr.Button("再次測驗")
            btn_back = gr.Button("回到設定")

    debug = gr.TextArea(label="Debug", visible=False)
    # endregion

    # region register events
    btn_start.click(
        reset,
        outputs=[n_correct, n_total, txt_accuracy, txt_info, txt_records],
        show_progress="hidden",
    ).then(
        start_test,
        [chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        [txt_test, st_queue, debug, tabs],
        show_progress="hidden",
    )

    txt_input.submit(
        check_answer,
        [txt_test, txt_input, n_correct, n_total, txt_records],
        [txt_input, txt_info, n_correct, n_total, txt_accuracy, txt_records],
        show_progress="hidden",
    ).then(
        next_char,
        [st_queue, n_correct, n_total, txt_records],
        [txt_test, st_queue, debug, txt_records, tabs],
        show_progress="hidden",
    )

    btn_select_all.click(
        select_all,
        outputs=[chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        show_progress="hidden",
    )

    btn_select_none.click(
        select_none,
        outputs=[chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        show_progress="hidden",
    )

    btn_again.click(
        reset,
        outputs=[n_correct, n_total, txt_accuracy, txt_info, txt_records],
        show_progress="hidden",
    ).then(
        start_test,
        [chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        [txt_test, st_queue, debug, tabs],
        show_progress="hidden",
    )

    btn_back.click(back, outputs=tabs, show_progress="hidden")
    # endregion

    app.launch(height=700)

重構版本

In [ ]:
def gr_args(fn=None, inputs=None, outputs=None, show_progress="hidden", **kwargs):
    return dict(fn=fn, inputs=inputs, outputs=outputs, show_progress=show_progress, **kwargs)

In [ ]:
font = gr.themes.GoogleFont("Kiwi Maru")
theme = gr.themes.Ocean(font=font)

with gr.Blocks(theme=theme) as app:
    st_queue = gr.State(None)

    # region define layout
    with gr.Tabs(selected=0) as tabs:
        with gr.Tab(label="設定", id=0):
            with gr.Group():
                chk_kana = gr.CheckboxGroup(["平假名", "片假名"], value=["平假名"], label="假名")
                chk_seion = gr.CheckboxGroup(category["seion"], value=["a"], label="清音")
                with gr.Row():
                    chk_dakuon = gr.CheckboxGroup(category["dakuon"], label="濁音")
                    chk_handakuon = gr.CheckboxGroup(category["handakuon"], label="半濁音")
                chk_youon = gr.CheckboxGroup(category["youon"], label="拗音")

            with gr.Row():
                btn_select_all = gr.Button("全選")
                btn_select_none = gr.Button("全不選")
            btn_start = gr.Button("開始測驗")

        with gr.Tab(label="測驗", id=1):
            with gr.Group():
                with gr.Row():
                    txt_test = gr.Textbox(label="題目", interactive=False)
                    txt_info = gr.Textbox(label="資訊", interactive=False)

                txt_input = gr.Textbox(label="作答", submit_btn=True)

                with gr.Row():
                    n_correct = gr.Number(label="答對題數", interactive=False)
                    n_total = gr.Number(label="總答題數", interactive=False)
                    txt_accuracy = gr.Text("100.00%", label="答對比率", interactive=False)

        with gr.Tab(label="紀錄", id=2):
            txt_records = gr.TextArea(show_label=False, interactive=False)
            btn_again = gr.Button("再次測驗")
            btn_back = gr.Button("回到設定")

    debug = gr.TextArea(label="Debug", visible=False)
    # endregion

    # region define arguments
    reset_outputs = [n_correct, n_total, txt_accuracy, txt_info, txt_records]
    reset_args = gr_args(reset, outputs=reset_outputs)

    start_inputs = [chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon]
    start_outputs = [txt_test, st_queue, debug, tabs]
    start_args = gr_args(start_test, start_inputs, start_outputs)

    check_inputs = [txt_test, txt_input, n_correct, n_total, txt_records]
    check_outputs = [txt_input, txt_info, n_correct, n_total, txt_accuracy, txt_records]
    check_args = gr_args(check_answer, check_inputs, check_outputs)

    next_inputs = [st_queue, n_correct, n_total, txt_records]
    next_outputs = [txt_test, st_queue, debug, txt_records, tabs]
    next_args = gr_args(next_char, next_inputs, next_outputs)

    select_all_outputs = [chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon]
    select_all_args = gr_args(select_all, outputs=select_all_outputs)

    select_none_outputs = [chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon]
    select_none_args = gr_args(select_none, outputs=select_none_outputs)

    back_args = gr_args(back, outputs=tabs)
    # endregion

    # region register events
    btn_start.click(**reset_args).then(start_args)
    txt_input.submit(**check_args).then(**next_args)
    btn_select_all.click(**select_all_args)
    btn_select_none.click(**select_none_args)
    btn_again.click(**reset_args).then(**start_args)
    btn_back.click(**back_args)
    # endregion

    app.launch(height=700)